In [1]:
import numpy as np
import gym
from stable_baselines3 import PPO
import math
import itertools
import datetime
import ray
import ray.rllib as rllib
from ray import tune
from ray.tune.registry import register_env

In [2]:
N = 13
rand = np.random.RandomState(1)
M = np.round(rand.normal(5, 1.5, size=(N, N)),2)
M[np.eye(N)==1] = 0
M

array([[0.  , 4.08, 4.21, 3.39, 6.3 , 1.55, 7.62, 3.86, 5.48, 4.63, 7.19,
        1.91, 4.52],
       [4.42, 0.  , 3.35, 4.74, 3.68, 5.06, 5.87, 3.35, 6.72, 6.35, 5.75,
        6.35, 3.97],
       [4.82, 3.6 , 0.  , 5.8 , 3.96, 4.4 , 3.97, 3.73, 3.99, 4.98, 3.32,
        5.35, 7.49],
       [6.11, 4.71, 3.67, 0.  , 7.54, 5.08, 4.04, 5.29, 8.15, 5.18, 5.93,
        5.45, 4.47],
       [3.29, 4.48, 4.69, 5.88, 0.  , 6.4 , 5.43, 6.33, 3.87, 6.88, 5.77,
        4.55, 5.73],
       [4.89, 6.7 , 7.28, 8.28, 2.91, 0.  , 4.24, 5.24, 6.31, 5.47, 1.97,
        4.54, 6.24],
       [5.35, 6.14, 4.67, 4.7 , 5.28, 5.62, 0.  , 5.18, 3.99, 5.57, 5.18,
        6.69, 6.8 ],
       [5.28, 4.44, 4.04, 5.64, 5.12, 4.48, 5.07, 0.  , 6.05, 4.33, 6.84,
        5.61, 5.89],
       [3.36, 5.25, 6.11, 3.57, 4.6 , 5.05, 2.94, 5.47, 0.  , 3.71, 5.53,
        3.03, 4.94],
       [2.58, 6.68, 5.61, 4.96, 3.84, 6.91, 7.95, 2.21, 6.85, 0.  , 5.51,
        3.2 , 6.3 ],
       [4.73, 4.09, 3.15, 5.83, 6.19, 4.06, 5.78, 

In [3]:
math.factorial(N-1)

479001600

In [4]:
class MyEnv(gym.Env):
    def __init__(self, env_config):
        super().__init__()
        self.n = N-1
        self.action_space = gym.spaces.Discrete(self.n)
        self.observation_space = gym.spaces.Dict({
            'visited': gym.spaces.MultiBinary(self.n), 
            'last': gym.spaces.Discrete(N)})   
        
    def reset(self):
        self.state = {'visited': np.zeros(self.n), 'last': 0}
        visited = np.zeros(self.n)
        return self.state

    def step(self, action):
        if self.state['visited'][action] == 1:
            self.reward = -10
        else:
            self.state['visited'][action] = 1
            self.reward = - M[self.state['last'], action + 1]
            self.state['last'] = action + 1
        if np.all(self.state['visited'] == 1): 
            self.reward += - M[action + 1, 0]    
            self.done = True
        else:
            self.done = False
            
        return self.state, self.reward, self.done, {}

In [5]:
config = rllib.agents.ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["framework"] = "torch"
config["env_config"] = {}

In [6]:
env = MyEnv(config)

In [7]:
print(datetime.datetime.now())
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=100000)
print(datetime.datetime.now())

2022-03-13 21:15:08.100306
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 39.7     |
|    ep_rew_mean     | -343     |
| time/              |          |
|    fps             | 2162     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 38.7        |
|    ep_rew_mean          | -333        |
| time/                   |             |
|    fps                  | 1569        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004341918 |
|    clip_fraction        | 0.00718     |
|    clip_range           | 0.2    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 29.5        |
|    ep_rew_mean          | -241        |
| time/                   |             |
|    fps                  | 1258        |
|    iterations           | 11          |
|    time_elapsed         | 17          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.006707578 |
|    clip_fraction        | 0.0377      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.45       |
|    explained_variance   | -0.000133   |
|    learning_rate        | 0.0003      |
|    loss                 | 1.91e+03    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0154     |
|    value_loss           | 4.66e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.9  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 16.8        |
|    ep_rew_mean          | -113        |
| time/                   |             |
|    fps                  | 1163        |
|    iterations           | 21          |
|    time_elapsed         | 36          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.012888955 |
|    clip_fraction        | 0.0926      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.18       |
|    explained_variance   | -3.1e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 567         |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0199     |
|    value_loss           | 1.09e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 15.1  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12.8        |
|    ep_rew_mean          | -69.6       |
| time/                   |             |
|    fps                  | 1052        |
|    iterations           | 31          |
|    time_elapsed         | 60          |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.017273413 |
|    clip_fraction        | 0.123       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.72       |
|    explained_variance   | -7.15e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 198         |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0205     |
|    value_loss           | 369         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12.6  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12.3        |
|    ep_rew_mean          | -58         |
| time/                   |             |
|    fps                  | 1040        |
|    iterations           | 41          |
|    time_elapsed         | 80          |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.012654748 |
|    clip_fraction        | 0.124       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.33       |
|    explained_variance   | -3.58e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 113         |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.0166     |
|    value_loss           | 252         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12.3  

In [8]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 11, 8, 6, 3, 2, 1, 12, 5, 10, 7, 9, 4], g = 42.35


In [10]:
x_min = []
g_min = 100000000
print(datetime.datetime.now())
for i, x in enumerate(itertools.permutations(range(1,N), N-1)):
    x = (0,) + x
    g = np.sum(M[x[:-1], x[1:]])+M[x[-1], x[0]]
    if g < g_min:
        g_min = g
        x_min = x
    if i % 10000000 == 0:
        print(i, x_min, g_min)
print(datetime.datetime.now())
print('Optimal solution:')
print(x_min, g_min)

2022-03-13 21:18:52.689905
0 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12) 66.36
10000000 (0, 1, 2, 10, 7, 9, 11, 8, 6, 3, 12, 5, 4) 44.720000000000006
20000000 (0, 1, 2, 10, 7, 9, 11, 8, 6, 3, 12, 5, 4) 44.720000000000006
30000000 (0, 1, 7, 9, 11, 8, 6, 3, 12, 5, 10, 2, 4) 44.29
40000000 (0, 1, 12, 5, 10, 7, 9, 11, 8, 6, 3, 2, 4) 44.24
50000000 (0, 1, 12, 5, 10, 7, 9, 11, 8, 6, 3, 2, 4) 44.24
60000000 (0, 1, 12, 5, 10, 7, 9, 11, 8, 6, 3, 2, 4) 44.24
70000000 (0, 1, 12, 5, 10, 7, 9, 11, 8, 6, 3, 2, 4) 44.24
80000000 (0, 1, 12, 5, 10, 7, 9, 11, 8, 6, 3, 2, 4) 44.24
90000000 (0, 3, 2, 1, 12, 5, 10, 7, 9, 11, 8, 6, 4) 43.769999999999996
100000000 (0, 3, 2, 1, 12, 5, 10, 7, 9, 11, 8, 6, 4) 43.769999999999996
110000000 (0, 3, 2, 1, 12, 5, 10, 7, 9, 11, 8, 6, 4) 43.769999999999996
120000000 (0, 3, 12, 5, 10, 7, 9, 11, 8, 6, 2, 1, 4) 43.67
130000000 (0, 3, 12, 5, 10, 7, 9, 11, 8, 6, 2, 1, 4) 43.67
140000000 (0, 3, 12, 5, 10, 7, 9, 11, 8, 6, 2, 1, 4) 43.67
150000000 (0, 3, 12, 5, 10, 7, 9, 11, 8,

In [11]:
ray.shutdown()
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:58271',
 'object_store_address': '/tmp/ray/session_2022-03-13_22-55-40_868274_25918/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-13_22-55-40_868274_25918/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-13_22-55-40_868274_25918',
 'metrics_export_port': 63337,
 'gcs_address': '127.0.0.1:61628',
 'node_id': 'eb5a4aa09f81895294a4836a4fbb293687e2ca732a65f10daa9e65a7'}

In [12]:
agent = rllib.agents.ppo.PPOTrainer(config=config, env=MyEnv)

2022-03-13 22:55:49,537	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-03-13 22:55:49,541	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=27233) 2022-03-13 22:55:59,272	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-03-13 22:55:59,339	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [13]:
print(datetime.datetime.now())
for i in range(31):
   # Perform one iteration of training the policy with PPO
   result = agent.train()
   if i % 10 == 0:
       #print(pretty_print(result))
       print('i: ', i)
       print('mean episode length:', result['episode_len_mean'])
       print('max episode reward:', result['episode_reward_max'])
       print('mean episode reward:', result['episode_reward_mean'])
       print('min episode reward:', result['episode_reward_min'])
       print('total episodes:', result['episodes_total'])
       print()

       checkpoint = agent.save()
print(datetime.datetime.now())

2022-03-13 22:55:59.366087


2022-03-13 22:56:10,926	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


i:  0
mean episode length: 37.556603773584904
max episode reward: -119.48
mean episode reward: -320.91037735849056
min episode reward: -913.9899999999999
total episodes: 106

i:  10
mean episode length: 13.23841059602649
max episode reward: -50.17
mean episode reward: -75.25076158940398
min episode reward: -120.49
total episodes: 2187

i:  20
mean episode length: 12.102719033232628
max episode reward: -43.88
mean episode reward: -55.2219335347432
min episode reward: -84.53999999999999
total episodes: 5438

i:  30
mean episode length: 12.024024024024024
max episode reward: -42.94
mean episode reward: -49.662912912912915
min episode reward: -71.28
total episodes: 8758

2022-03-13 23:01:26.211702


In [14]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action = agent.compute_single_action(obs, explore = False)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 11, 8, 9, 7, 1, 12, 3, 2, 6, 5, 10, 4], g = 45.61999999999999


In [15]:
def env_creator(env_config):
    return MyEnv(env_config)  # return an env instance

register_env("my_env", env_creator)

In [16]:
experiment = tune.run(
    rllib.agents.ppo.PPOTrainer,
    config={
        "env": "my_env",
        "num_gpus": 0,
        "num_workers": 1,
        "framework": "torch"
    },
    metric="episode_reward_mean",
    mode="max",
    stop={"training_iteration": 50},
    checkpoint_at_end=True
)

Trial name,status,loc
PPOTrainer_my_env_5d5b7_00000,PENDING,


(PPOTrainer pid=27234) 2022-03-13 23:01:35,673	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=27234) 2022-03-13 23:01:35,674	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
PPOTrainer_my_env_5d5b7_00000,RUNNING,127.0.0.1:27234


(PPOTrainer pid=27234) 2022-03-13 23:01:44,002	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=27232) 2022-03-13 23:01:43,974	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!


Trial name,status,loc
PPOTrainer_my_env_5d5b7_00000,RUNNING,127.0.0.1:27234


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_5d5b7_00000,RUNNING,127.0.0.1:27234,49,490.447,196000,-47.1763,-42.07,-63.95,12.018


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_5d5b7_00000,RUNNING,127.0.0.1:27234,49,490.447,196000,-47.1763,-42.07,-63.95,12.018


Result for PPOTrainer_my_env_5d5b7_00000:
  agent_timesteps_total: 200000
  custom_metrics: {}
  date: 2022-03-13_23-10-06
  done: true
  episode_len_mean: 12.03003003003003
  episode_media: {}
  episode_reward_max: -42.690000000000005
  episode_reward_mean: -50.06555555555556
  episode_reward_min: -73.18
  episodes_this_iter: 333
  episodes_total: 15096
  experiment_id: 3957756f55bc4d9a8b08a7cd6eb29e57
  hostname: MacBook-Pro-Vladimir.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.7593750000000002
          cur_lr: 5.0000000000000016e-05
          entropy: 0.7507079626283338
          entropy_coeff: 0.0
          kl: 0.010414600522083716
          policy_loss: -0.02518911186645749
          total_loss: 167.40628174812562
          vf_explained_var: -3.0763687626008064e-09
          vf_loss: 167.423561457152
        model: {}
    num_agent_steps_sampled: 200000
    num_agent_s

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_5d5b7_00000,TERMINATED,127.0.0.1:27234,50,501.084,200000,-50.0656,-42.69,-73.18,12.03


(PPOTrainer pid=27234) 2022-03-13 23:10:06,164	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!
(RolloutWorker pid=27232) 2022-03-13 23:10:06,751	ERROR worker.py:432 -- SystemExit was raised from the worker.
(RolloutWorker pid=27232) Traceback (most recent call last):
(RolloutWorker pid=27232)   File "python/ray/_raylet.pyx", line 629, in ray._raylet.execute_task
(RolloutWorker pid=27232)   File "python/ray/_raylet.pyx", line 636, in ray._raylet.execute_task
(RolloutWorker pid=27232)   File "python/ray/_raylet.pyx", line 640, in ray._raylet.execute_task
(RolloutWorker pid=27232)   File "python/ray/_raylet.pyx", line 589, in ray._raylet.execute_task.function_executor
(RolloutWorker pid=27232)   File "/Users/vladimirsudakov/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ray/_private/function_manager.py", line 639, in actor_method_executor
(RolloutWorker pid=27232)     retu

In [17]:
agent.restore(experiment.best_checkpoint)

2022-03-13 23:10:06,905	INFO trainable.py:472 -- Restored on 127.0.0.1 from checkpoint: /Users/vladimirsudakov/ray_results/PPOTrainer_2022-03-13_23-01-26/PPOTrainer_my_env_5d5b7_00000_0_2022-03-13_23-01-26/checkpoint_000050/checkpoint-50
2022-03-13 23:10:06,906	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 50, '_timesteps_total': 200000, '_time_total': 501.08385014533997, '_episodes_total': 15096}


In [18]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action = agent.compute_single_action(obs, explore = False)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 5, 10, 7, 1, 12, 3, 2, 4, 11, 8, 6, 9], g = 43.15
